In [1]:
import getpass
import os

In [24]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [57]:
! pip install python-docx

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 25.2 MB/s eta 0:00:00


In [59]:
from langchain.document_loaders import Docx2txtLoader

# Initialize the loader with your Word document file path
loader = Docx2txtLoader("resume.docx")
docs = loader.load()

In [60]:
docs

[Document(metadata={'source': 'resume.docx'}, page_content='Kaarthicsudhan Mani\n\n+1 (407) - 901 - 7892 | smk015930@gmail.com\n\nProfessional Summary\n\n\tOver 7 years of hands-on experience in developing high-scale Python backend applications with a focus on e-commerce and retail tech platforms, delivering secure and efficient solutions to meet business needs.\n\n\tProficient in containerization (Docker) and orchestration (Kubernetes) to deploy and manage microservices, ensuring high availability, scalability, and seamless performance during high-traffic periods.\n\n\tSkilled in designing and implementing CI/CD pipelines using tools like Jenkins, GitLab CI, and Azure DevOps to automate build, test, and deployment processes, reducing manual intervention and improving release cycles.\n\n\tExpert in building RESTful APIs for data-intensive platforms, optimizing response times through efficient data handling and asynchronous processing to achieve sub-200ms latency in high-load scenarios.

In [61]:
print(docs[0].page_content[:500])

Kaarthicsudhan Mani

+1 (407) - 901 - 7892 | smk015930@gmail.com

Professional Summary

	Over 7 years of hands-on experience in developing high-scale Python backend applications with a focus on e-commerce and retail tech platforms, delivering secure and efficient solutions to meet business needs.

	Proficient in containerization (Docker) and orchestration (Kubernetes) to deploy and manage microservices, ensuring high availability, scalability, and seamless performance during high-traffic periods


In [62]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 18 sub-documents.


In [97]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Split the document into chunks
chunks = text_splitter.split_documents(docs)

# Print the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk.page_content}\n")


Chunk 1:
Kaarthicsudhan Mani

+1 (407) - 901 - 7892 | smk015930@gmail.com

Professional Summary

	Over 7 years of hands-on experience in developing high-scale Python backend applications with a focus on e-commerce and retail tech platforms, delivering secure and efficient solutions to meet business needs.

Chunk 2:
Proficient in containerization (Docker) and orchestration (Kubernetes) to deploy and manage microservices, ensuring high availability, scalability, and seamless performance during high-traffic periods.

	Skilled in designing and implementing CI/CD pipelines using tools like Jenkins, GitLab CI, and Azure DevOps to automate build, test, and deployment processes, reducing manual intervention and improving release cycles.

Chunk 3:
Expert in building RESTful APIs for data-intensive platforms, optimizing response times through efficient data handling and asynchronous processing to achieve sub-200ms latency in high-load scenarios.

	Experienced in integrating computer vision (CV) 

In [107]:
print(chunks)

[Document(metadata={'section': 'Professional Summary'}, page_content='Kaarthicsudhan Mani\n\n+1 (407) - 901 - 7892 | smk015930@gmail.com\n\nProfessional Summary\n\n\tOver 7 years of hands-on experience in developing high-scale Python backend applications with a focus on e-commerce and retail tech platforms, delivering secure and efficient solutions to meet business needs.'), Document(metadata={'section': 'General'}, page_content='Proficient in containerization (Docker) and orchestration (Kubernetes) to deploy and manage microservices, ensuring high availability, scalability, and seamless performance during high-traffic periods.\n\n\tSkilled in designing and implementing CI/CD pipelines using tools like Jenkins, GitLab CI, and Azure DevOps to automate build, test, and deployment processes, reducing manual intervention and improving release cycles.'), Document(metadata={'section': 'General'}, page_content='Expert in building RESTful APIs for data-intensive platforms, optimizing response 

In [99]:

# Step 3: Define a function to assign custom metadata based on section titles
def assign_metadata(chunk):
    text = chunk.page_content.lower()

    if "professional summary" in text:
        return {"section": "Professional Summary"}
    elif "environment and tools" in text:
        return {"section": "Environment and Tools"}
    elif "professional experience" in text:
        return {"section": "Professional Experience"}
    elif "education" in text:
        return {"section": "Education"}
    else:
        return {"section": "General"}

# Step 4: Assign metadata to each chunk
for chunk in chunks:
    chunk.metadata = assign_metadata(chunk)

# Step 5: Print the chunks with metadata
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1} Metadata: {chunk.metadata}")
    print(f"Chunk Content:\n{chunk.page_content}\n")

Chunk 1 Metadata: {'section': 'Professional Summary'}
Chunk Content:
Kaarthicsudhan Mani

+1 (407) - 901 - 7892 | smk015930@gmail.com

Professional Summary

	Over 7 years of hands-on experience in developing high-scale Python backend applications with a focus on e-commerce and retail tech platforms, delivering secure and efficient solutions to meet business needs.

Chunk 2 Metadata: {'section': 'General'}
Chunk Content:
Proficient in containerization (Docker) and orchestration (Kubernetes) to deploy and manage microservices, ensuring high availability, scalability, and seamless performance during high-traffic periods.

	Skilled in designing and implementing CI/CD pipelines using tools like Jenkins, GitLab CI, and Azure DevOps to automate build, test, and deployment processes, reducing manual intervention and improving release cycles.

Chunk 3 Metadata: {'section': 'General'}
Chunk Content:
Expert in building RESTful APIs for data-intensive platforms, optimizing response times through e

In [115]:
print(chunk.page_content)

University of Central Florida - Masters in Computer Science                                                                        August 2017


In [100]:
from langchain_huggingface import HuggingFaceEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore

In [101]:
from pinecone import Pinecone, PodSpec
from pinecone import ServerlessSpec, PodSpec  
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)  

index = pc.Index("test")  
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 18}},
 'total_vector_count': 18}

In [102]:
# Set the Pinecone API key directly in the environment variable
# os.environ['PINECONE_API_KEY'] = getpass.getpass("Enter Pinecone API key: ")

# index_name = "chatbot-rag"  # Name of the Pinecone index
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [68]:
all_splits[0].page_content

'Kaarthicsudhan Mani\n\n+1 (407) - 901 - 7892 | smk015930@gmail.com\n\nProfessional Summary\n\n\tOver 7 years of hands-on experience in developing high-scale Python backend applications with a focus on e-commerce and retail tech platforms, delivering secure and efficient solutions to meet business needs.\n\n\tProficient in containerization (Docker) and orchestration (Kubernetes) to deploy and manage microservices, ensuring high availability, scalability, and seamless performance during high-traffic periods.\n\n\tSkilled in designing and implementing CI/CD pipelines using tools like Jenkins, GitLab CI, and Azure DevOps to automate build, test, and deployment processes, reducing manual intervention and improving release cycles.\n\n\tExpert in building RESTful APIs for data-intensive platforms, optimizing response times through efficient data handling and asynchronous processing to achieve sub-200ms latency in high-load scenarios.'

In [111]:
# embeddings.embed_query(all_splits[0].page_content)
document_embeddings = [embeddings.embed_query(doc.page_content) for doc in chunks]
document_embeddings

[[0.05776212736964226,
  -0.02980569563806057,
  -0.02390344627201557,
  0.001525171916000545,
  0.008746094070374966,
  -0.013705337420105934,
  0.06797704100608826,
  -0.023235829547047615,
  -0.008690356276929379,
  -0.022406302392482758,
  0.04474873095750809,
  0.03334324061870575,
  0.025825895369052887,
  0.13912439346313477,
  -0.004972927272319794,
  -0.004409558605402708,
  -0.016764521598815918,
  0.00448177196085453,
  -0.014940032735466957,
  0.03355437144637108,
  -0.06513641029596329,
  0.040665656328201294,
  -0.027707722038030624,
  0.022771969437599182,
  -0.018605059012770653,
  -0.034720223397016525,
  -0.009152097627520561,
  0.007437569554895163,
  -0.011151020415127277,
  -0.06606490164995193,
  0.03933039680123329,
  -0.024483641609549522,
  -0.004262005444616079,
  0.009126056917011738,
  2.519434019632172e-06,
  -0.028115276247262955,
  -0.011060808785259724,
  -0.006596939638257027,
  -0.06031252443790436,
  0.0872650146484375,
  0.0279180109500885,
  0.05157

In [70]:
len(document_embeddings[0]) 

768

In [71]:
print(document_embeddings[0])

[0.035575587302446365, -0.005477422382682562, -0.048231951892375946, -0.03100946731865406, 0.006611694116145372, -0.015398913063108921, 0.038490332663059235, 0.0015491503290832043, -0.01064416952431202, 0.013294002041220665, 0.05767811834812164, 0.04549326375126839, 0.01308590266853571, 0.12534891068935394, -0.00792810320854187, -0.019483882933855057, 0.005065251607447863, -0.00426966417580843, -0.025983110070228577, 0.029447097331285477, -0.04295482859015465, 0.010068578645586967, -0.013504219241440296, 0.005674656480550766, -0.018524330109357834, -0.04391806200146675, 0.0046319011598825455, 0.028739742934703827, -0.02502565085887909, -0.061163529753685, 0.027363650500774384, -0.016613595187664032, -0.010952485725283623, 0.02003452740609646, 2.304176177858608e-06, -0.007191381882876158, 0.010127948597073555, 0.018346372991800308, -0.03272968903183937, 0.0439281165599823, 0.05426280200481415, -0.019236057996749878, 0.0091702900826931, 0.04262178763747215, -0.004439610056579113, 0.00207

In [72]:
upsert_data = [
    (str(i), embedding) for i, embedding in enumerate(document_embeddings)
]

In [73]:
print(upsert_data[0])

('0', [0.035575587302446365, -0.005477422382682562, -0.048231951892375946, -0.03100946731865406, 0.006611694116145372, -0.015398913063108921, 0.038490332663059235, 0.0015491503290832043, -0.01064416952431202, 0.013294002041220665, 0.05767811834812164, 0.04549326375126839, 0.01308590266853571, 0.12534891068935394, -0.00792810320854187, -0.019483882933855057, 0.005065251607447863, -0.00426966417580843, -0.025983110070228577, 0.029447097331285477, -0.04295482859015465, 0.010068578645586967, -0.013504219241440296, 0.005674656480550766, -0.018524330109357834, -0.04391806200146675, 0.0046319011598825455, 0.028739742934703827, -0.02502565085887909, -0.061163529753685, 0.027363650500774384, -0.016613595187664032, -0.010952485725283623, 0.02003452740609646, 2.304176177858608e-06, -0.007191381882876158, 0.010127948597073555, 0.018346372991800308, -0.03272968903183937, 0.0439281165599823, 0.05426280200481415, -0.019236057996749878, 0.0091702900826931, 0.04262178763747215, -0.004439610056579113, 0

In [74]:
index.upsert(upsert_data)

{'upserted_count': 18}

In [ ]:
# query_embedding = embeddings.embed_query(query)
# results = index.query(query_embedding, top_k=5, include_metadata=True)
# retrieved_texts = [match['metadata'].get('text', '') for match in results['matches']]
# context = "\n".join(retrieved_texts)
# return context

In [83]:
# Example query
query = "Who is Kaarthicsudhan and how many year experience he has?"

# Convert the query to an embedding using the same embedding model
query_embedding = embeddings.embed_query(query)

In [84]:
query_embedding

[0.03689425811171532,
 0.0022931979037821293,
 0.01142441388219595,
 0.005410489160567522,
 -0.02761772833764553,
 -0.041779834777116776,
 0.020474854856729507,
 0.017264075577259064,
 -0.052643775939941406,
 -0.00402256241068244,
 0.03842249885201454,
 0.031577903777360916,
 0.015545387752354145,
 0.005365916062146425,
 0.009567118249833584,
 0.024479147046804428,
 0.02483293041586876,
 0.028822509571909904,
 -0.020138703286647797,
 -0.019704129546880722,
 -0.021921392530202866,
 -0.056767478585243225,
 -0.08550866693258286,
 0.07092663645744324,
 -0.047598302364349365,
 -0.0603032223880291,
 0.029488390311598778,
 -0.004366923123598099,
 -0.024044321849942207,
 0.001674781204201281,
 0.02262154221534729,
 -0.013109819032251835,
 -0.05234166607260704,
 0.0014025490963831544,
 1.1229302572246525e-06,
 -0.000289121235255152,
 0.0291384756565094,
 -0.019951313734054565,
 0.01184695865958929,
 0.0011012685718014836,
 -0.0071388655342161655,
 0.05532537400722504,
 0.007177757099270821,
 -0

In [85]:
# Perform a similarity search in Pinecone
results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

# Print the results
for match in results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}, Metadata: {match.get('metadata', {})}")

ID: 2, Score: 0.401025891, Metadata: {}
ID: 3, Score: 0.353546888, Metadata: {}
ID: 10, Score: 0.267026603, Metadata: {}


In [96]:
print(results)

{'matches': [{'id': '2', 'score': 0.401025891, 'values': []},
             {'id': '3', 'score': 0.353546888, 'values': []},
             {'id': '10', 'score': 0.267026603, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [95]:
results

{'matches': [{'id': '2', 'score': 0.401025891, 'values': []},
             {'id': '3', 'score': 0.353546888, 'values': []},
             {'id': '10', 'score': 0.267026603, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [86]:
retrieved_texts = [match['values'] for match in results['matches']]

In [90]:
retrieved_texts

[[], [], []]

In [92]:
# Combine the retrieved texts to form the context
context = "\n".join([" ".join(text) for text in retrieved_texts])

# Create the prompt with the retrieved context
prompt = f"""
Context:
{context}

Question:
{query}
"""

In [93]:
prompt

'\nContext:\n\n\n\n\nQuestion:\nWho is Kaarthicsudhan and how many year experience he has?\n'

In [94]:
# Get the response from the LLM
response = llm.invoke([{"role": "user", "content": prompt}])

# Print the response
print("LLM Response:\n", response.content)

LLM Response:
 I apologize, but there is no context provided about Kaarthicsudhan or his experience. Can you please provide more information or clarify who Kaarthicsudhan is and what kind of experience you are referring to? I'll do my best to help.
